In [1]:
## transferring code previously contained in the nebulously named RDA_hov_periods to its own notebook explaining
## how Hovmoller occupancy plots of intensity were constructed.

In [1]:
import numpy as np
import time
import os
import netCDF4 as nc
import datetime
import matplotlib.pyplot as plt
%matplotlib notebook

In [2]:
#Access NetCDF files (currently hosted in Ferret directory)
RDA_path_1 = "/Users/Siwen/Desktop/ferret/bin/meiyu_clean.nc"
RDA_path_2 = "/Users/Siwen/Desktop/ferret/bin/meiyu_2_clean.nc"
RDA_1 = nc.Dataset(RDA_path_1, 'r')
RDA_2 = nc.Dataset(RDA_path_2, 'r')

In [3]:
#load data from NetCDF files to notebook
lat_1_all =  RDA_1.variables['lat_115'][:]
lat_2_all =  RDA_2.variables['lat_115'][:]
intensity_1_all = RDA_1.variables['intensity'][:] 
intensity_2_all = RDA_2.variables['intensity'][:]

RDA_1.close()
RDA_2.close()

In [4]:
#Assign a calendar date to each time point
startday = datetime.datetime(1951,1,1)
date_list = np.array([datetime.timedelta(days=x) + startday for x in range(0, 20819)])

In [5]:
#Define empty variables reshaped with axis 0 as day of year and axis 1 as year.

#time periods of interest - period1 (p1) and period2 (p2)
#adjustment: mymap (below) now also restricts by years of interest.
yr_beg = [1951,1951,1980,1980,1994]
yr_end = [2007,1979,2007,1993,2007]

#variable storage for period 1 and period 2 - initialized as e
lats_1 = []
lats_2 = []
intensity_1 = []
intensity_2 = []


## TIME PERIOD 1 - arrange data by day of year and then year
for yr_b, yr_e in zip(yr_beg, yr_end):

    lat_1 = np.zeros([365, yr_e-yr_b+1])
    lat_2 = np.zeros([365, yr_e-yr_b+1])
    int_1 = np.zeros([365, yr_e-yr_b+1])
    int_2 = np.zeros([365, yr_e-yr_b+1])
        
    for i, date in enumerate(date_list[0:365]): #go through each day of the year
    
        dd = date.day
        mm = date.month
        mymap = map(lambda date: (date.day == dd) & (date.month == mm) & (date.year >= yr_b) & (date.year <= yr_e), date_list)
        extract = np.array(list(mymap))
    
        lat_1[i,:] = lat_1_all[extract]
        lat_2[i,:] = lat_2_all[extract]
        int_1[i,:] = intensity_1_all[extract]
        int_2[i,:] = intensity_2_all[extract]
             
    lats_1.append(lat_1)
    lats_2.append(lat_2)
    intensity_1.append(int_1)
    intensity_2.append(int_2)

In [6]:
#quick function that bins everything in a 2D array and puts floor and ceiling on array values.
def findbins(lat):
    index = np.ndarray.round(lat-19.5)
    index[index < 0] = 0
    index[index > 21] = 21
    return index

In [8]:
## find bins corresponding to latitude of each event
lats_1_indices = [ findbins(x) for x in lats_1 ]
lats_2_indices = [ findbins(x) for x in lats_2 ]

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:4: RuntimeWarning: invalid value encountered in less
//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: RuntimeWarning: invalid value encountered in greater


In [9]:
## INTENSITY HOVMOLLER PLOT ##
#First step - convert to useful format. 3-D numpy array where the first dimension corresponds to day of the year, 
#the second latitude bin and the third to different instancea on the same day in same bin.
#Code automatically expands the array in 3rd dimension when necessary.

## same years of interest as above:
#yr_beg = [1951,1951,1980,1980,1994]
#yr_end = [2007,1979,2007,1993,2007]

#initialize Hovmoller storage for intensity
ints_1_hov = []

#strategy below - for each day, fill in new np array of occupancy. expand the ensemble dimension as needed.
#then, in main loop, expand the occupancy array in the ensemble direction if necessary based on contents of subloop.

##PRIMARY EVENTS
for myints, mylats in zip(intensity_1, lats_1_indices):
    
    #initialize hovmoller storage for this loop
    my_int_hov = np.empty([365,22,10]) #from experience with this data set, at most 10 events at same latitude
    my_int_hov[:] = np.NAN
    
    for dd, col in enumerate(mylats):

        #print(dd)

        #for this particular day, will store binned intensities for each latitude.
        int_daily = np.empty([22,10])
        int_daily[:] = np.NAN

        for j, index in enumerate(col):
            if not np.isnan(index):

                next_empty = np.where(np.isnan(int_daily[index,:]))[0][0]                
                int_daily[index,next_empty] = myints[dd,j] #append next intensity value to existing values at given bin

        #following assignment is implicitly from 0 up until the first reclen columns
        my_int_hov[dd,:,:] = int_daily
    
    ints_1_hov.append(my_int_hov)

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:34: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:35: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [13]:
## initialize storage
ints_2_hov = []

##SECONDARY EVENTS
for myints, mylats in zip(intensity_2, lats_2_indices):
    
    #initialize hovmoller storage for this loop
    my_int_hov = np.empty([365,22,10]) #from experience with this data set, at most 10 events at same latitude
    my_int_hov[:] = np.NAN
    
    for dd, col in enumerate(mylats):

        #print(dd)

        #for this particular day, will store binned intensities for each latitude.
        int_daily = np.empty([22,10])
        int_daily[:] = np.NAN
        
        for j, index in enumerate(col):
            if not np.isnan(index):

                next_empty = np.where(np.isnan(int_daily[index,:]))[0][0]                
                int_daily[index,next_empty] = myints[dd,j] #append next intensity value to existing values at given bin

        #following assignment is implicitly from 0 up until the first reclen columns
        my_int_hov[dd,:,:] = int_daily
        
    ints_2_hov.append(my_int_hov)

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:22: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:23: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [14]:
## JOINT INTENSITY HOVMOLLER - combining both primary and secondary (not sure if will end up using or not)
ints_hov = []

for int1,int2 in zip(ints_1_hov, ints_2_hov):
    
    my_int_hov = np.empty([365,22,10])

    for dd, (hov_1, hov_2) in enumerate(zip(int1,int2)):

        #print(dd)

        hov_daily = np.empty([22,10])
        hov_daily[:] = np.NAN

        for j, (rec1, rec2) in enumerate(zip(hov_1,hov_2)):
            
            ## fuse primary and secondary events, lat bin by lat bin
            r1 = rec1[~np.isnan(rec1)]
            r2 = rec2[~np.isnan(rec2)]
            rec = np.append(r1,r2)
            hov_daily[j,:len(rec)] = rec

        my_int_hov[dd,:,:] = hov_daily
    
    ints_hov.append(my_int_hov)

In [15]:
## outputs the intensity of each event for each day, binned by latitude - useful for future bootstrapping efforts

#SAVE INTENSITY HOVMOLLER AS NETCDF FILE
savefile = "RDA_int_hov"
ferretpath = "/Users/Siwen/Desktop/Ferret/bin/"

#netCDF output file formatting formatting
hovname = ferretpath + savefile + ".nc"

#check if file exists and if so destroy it!
if os.path.isfile(hovname):
    os.remove(hovname)

#create NetCDF file
inthovout = nc.Dataset(hovname, "w")

#create dimensions in NetCDF file
days_dim = inthovout.createDimension('time', None)
lat_dim = inthovout.createDimension('lat', 22)
rec_dim = inthovout.createDimension('records', 10)
dim_tuple = ("time","lat","records")

#use same years as chosen above:
#yr_beg = [1951,1951,1980,1980,1994]
#yr_end = [2007,1979,2007,1993,2007]

for i, (yr_b, yr_e) in enumerate(zip(yr_beg, yr_end)):
    
    #convoluted line of code below guarantees that 2007 will become '07' and not '7'
    years = str(yr_b-1900) + str("{0:0=2d}".format((yr_e-1900)%100))
    print(years)

    #create variables inside netCDF file
    int_1_out = inthovout.createVariable("int_1_hov_" + years, 'f8', dim_tuple)
    int_2_out = inthovout.createVariable("int_2_hov_" + years, 'f8', dim_tuple)
    int_out = inthovout.createVariable("int_hov_" + years, 'f8', dim_tuple)

    int_1_out[:] = ints_1_hov[i]
    int_2_out[:] = ints_2_hov[i]
    int_out[:] = ints_hov[i]

inthovout.close()

5107
5179
8007
8093
9407
